In [8]:
from scipy.optimize import NonlinearConstraint, minimize, LinearConstraint
import numpy as np
from scipy.special import gamma, factorial


In [10]:
h = -1.50
N = 50
gamma_const = 0.5
delta_const = 1.0
T = 1
X = 0.1
G = np.zeros((N, N))

def G_ij_upper(i, j):
    return (1 / (1 - gamma_const) * (1 - gamma_const)) * (T / N) ** (2 - gamma_const) \
            * ((i - j + 1) ** (2 - gamma_const) - 2 * (i - j) ** (2 - gamma_const) +\
                 (i - j - 1) ** (2 - gamma_const)) 

def G_ii_diagonal():
    return (2 / ((1 - gamma_const) * (2 - gamma_const))) * (T / N) ** (2 - gamma_const)
    
G_upper = np.fromfunction(G_ij_upper, (N, N))
G_upper = np.nan_to_num(G_upper, nan = 0.0)
G_diag = np.repeat(G_ii_diagonal(), N)
G_diag = np.diag(G_diag)
G = G_upper.T + G_upper + G_diag

v_guess = np.repeat(T * X / N, N) ## VWAP

def gss_guess(t: np.array) -> np.array:
    gamma_1 = gamma((1 + gamma_const)/ 2)
    gamma_2 = gamma(1 + (gamma_const / 2))
    c = (X / N) / (np.sqrt(np.pi) * (T / 2) ** gamma_const * (gamma_1 / gamma_2))
    return (c / (t * (T - t)) ** ((1 - gamma_const) / 2))
gss_v_0_guess = gss_guess(np.arange(0, 1 + 1 / (N + 1), 1 / (N + 1)))[1:-1]

<ipython-input-10-851712020c2d>:11: RuntimeWarning: invalid value encountered in power
  * ((i - j + 1) ** (2 - gamma_const) - 2 * (i - j) ** (2 - gamma_const) +\
<ipython-input-10-851712020c2d>:12: RuntimeWarning: invalid value encountered in power
  (i - j - 1) ** (2 - gamma_const))
<ipython-input-10-851712020c2d>:29: RuntimeWarning: divide by zero encountered in true_divide
  return (c / (t * (T - t)) ** ((1 - gamma_const) / 2))


In [11]:
def impact_f(v):
    return np.sign(v) * np.abs(v) ** delta_const

A = G_upper + G_diag / 2

def expect_cost_compute(v):
    return v @ A @ impact_f(v).T

In [12]:
def total_volume(x):
    return np.sum(x)

lc = LinearConstraint(total_volume, N * X / T, N * X / T)

In [ ]:
minimize(expect_cost_compute, v_guess, method = "SLSQP", constraints=(lc))